In [1]:
# Import necessary libraries
import os
import sys
import argparse
from pathlib import Path
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, models
from model import CombinedModel    
from datasets import get_data_loaders
from train import train
#from test import evaluate, load_model

# from tensorboard_utils import setup_tensorboard

In [2]:
def parse_args():
    parser = argparse.ArgumentParser(description='Training script')
    parser.add_argument('--learning_rate', type=float, default=1e-3, help='Learning rate for the optimizer')
    parser.add_argument('--checkpoint_dir', type=str, default='./checkpoints', help='Directory to save checkpoints')
    parser.add_argument('--log_dir', type=str, default='./logs', help='Directory for TensorBoard logs')
    parser.add_argument('--train_dir', type=str, default='D:/Projects/SP CUP Dataset/valid', help='Directory for training data')
    parser.add_argument('--test_dir', type=str, default='D:/Projects/SP CUP Dataset/valid', help='Directory for testing data')
    parser.add_argument('--batch_size', type=int, default=16, help='Batch size for training')
    parser.add_argument('--epochs', type=int, default=2, help='Number of epochs to train')
    
    # Use parse_known_args to avoid errors
    args, unknown = parser.parse_known_args()
    return args


args = parse_args()

os.makedirs(args.checkpoint_dir, exist_ok=True)
os.makedirs(args.log_dir, exist_ok=True)

print(args)


Namespace(learning_rate=0.001, checkpoint_dir='./checkpoints', log_dir='./logs', train_dir='D:/Projects/SP CUP Dataset/valid', test_dir='D:/Projects/SP CUP Dataset/valid', batch_size=16, epochs=2)


In [3]:
# Setup TensorBoard
# writer = setup_tensorboard(args.log_dir)

### Initialize Model

In [4]:
# Initialize model
print("Initializing Model")
model = CombinedModel()

"""model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
num_features = model.fc.in_features  # Get the number of features from the current fc layer
model.fc = nn.Sequential(
    nn.Linear(num_features, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 1), # Output layer for binary classification (Fake/Real)
    nn.Sigmoid()
)"""

print(model)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"device: {device}")
model = model.to(device)

Initializing Model
CombinedModel(
  (freq_branch): FrequencyBranch(
    (fc1): Linear(in_features=98304, out_features=512, bias=True)
    (fc2): Linear(in_features=512, out_features=256, bias=True)
    (fc3): Linear(in_features=256, out_features=128, bias=True)
    (relu): ReLU()
  )
  (conv_branch): ConvBranch(
    (model): Sequential(
      (0): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): ReLU()
        (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (6): Dropout(p=0.3, inplace=False)
      )
      (1): Sequential(
        (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2):

### Loss Function and Optimizer

In [5]:
# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=args.learning_rate)

### Image Preprocessing

In [6]:
# Load datasets
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.3996, 0.3194, 0.3223], [0.2321, 0.1766, 0.1816])
])
train_loader, valid_loader = get_data_loaders(args.train_dir, args.test_dir, args.batch_size, transform)

### Train Model

In [ ]:
# Train the model
print("Start Training")
train(model, train_loader, criterion, optimizer, args.epochs, args.checkpoint_dir)

### Load Model

In [ ]:
loaded_model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

# Modify the fully connected layer to match the saved model
num_features = loaded_model.fc.in_features
loaded_model.fc = nn.Sequential(
	nn.Linear(num_features, 512),
	nn.ReLU(),
	nn.Dropout(0.5),
	nn.Linear(512, 1),
	nn.Sigmoid()
)

MODEL_SAVE_PATH = args.checkpoint_dir + '/final_checkpoint.pth'
state_dict = torch.load(MODEL_SAVE_PATH)

loaded_model.load_state_dict(state_dict)
loaded_model = loaded_model.to(device)
print(f"Model loaded from {MODEL_SAVE_PATH}")

In [ ]:
# Set both models to evaluation mode
model.eval()
loaded_model.eval()

# Compare the state dictionaries
def compare_models(model1, model2):
    model1_dict = model1.state_dict()
    model2_dict = model2.state_dict()
    
    for key in model1_dict:
        if not torch.equal(model1_dict[key], model2_dict[key]):
            return False
    return True

# Check if the parameters are identical
are_identical = compare_models(model, loaded_model)
print(f"Are the trained model and loaded model parameters identical? {are_identical}")

In [ ]:
#checkpoint_path = os.path.join(args.checkpoint_dir, 'final_checkpoint.pth')
#print(f"Loading model from {checkpoint_path}")
#model = load_model(checkpoint_path, model)

evaluate(loaded_model, valid_loader)